# Bike Rental Forecasting using LSTM based Network #


## Prerequisites

Please install the following packages before proceeding. Open command line interface from Azure Machine Learning Workbench App and execute the following commands:

- pip install tensorflow
- pip install keras
- pip install h5py
- pip uninstall azure-ml-api-sdk
- pip install azure-ml-api-sdk
- pip uninstall azure-cli-ml
- pip install azure-cli-ml


In [2]:
import numpy as np
from numpy import newaxis
import keras
import datetime
import random
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, TimeDistributed
from sklearn import metrics as mt

import pandas as pd
from keras import optimizers
import datetime

slidingwindow = 12
maxval_w = 1
maxval_pb = 1

Using TensorFlow backend.


## Data Preparation ##

In [3]:
def parse_data(df, window_sz):
    data = np.squeeze(np.reshape(df, newshape= [-1,1]), axis=1)
    
    init = np.zeros(window_sz)
    result=[]
    data = np.append(init,data)
    #print(data[0:34])
    
    for index in range(window_sz,len(data)):
        tmp=[]
        tmp = data[index-window_sz:index]
        #tmp = data[2:14]
        rsed_arr = tmp[::-1]
        #result.append(np.append([data[index]],tmp))
        result.append(rsed_arr)
    result=np.array(result)
    print(result.shape)
    
    return result

In [4]:
def load_data(fname):
    global noballs, slidingwindow, norm_wndw
    print("*** loading data ***")
    df = pd.read_csv(fname)

    data = np.array(df.iloc[:, 12])
    
    trainX = parse_data(data, slidingwindow)
    df = np.concatenate((trainX,df), axis=1)

    print("Data has size of ", data.shape)
    print("Sample data on how lag features look for the forecast:\n",trainX[0:20:,])
    return df

In [5]:
preppeddata=load_data("Regression_ Demand estimation.csv")

*** loading data ***
(17379, 12)
Data has size of  (17379,)
Sample data on how lag features look for the forecast:
 [[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [  16.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [  40.   16.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [  32.   40.   16.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [  13.   32.   40.   16.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   1.   13.   32.   40.   16.    0.    0.    0.    0.    0.    0.    0.]
 [   1.    1.   13.   32.   40.   16.    0.    0.    0.    0.    0.    0.]
 [   2.    1.    1.   13.   32.   40.   16.    0.    0.    0.    0.    0.]
 [   3.    2.    1.    1.   13.   32.   40.   16.    0.    0.    0.    0.]
 [   8.    3.    2.    1.    1.   13.   32.   40.   16.    0.    0.    0.]
 [  14.    8.    3.    2.    1.    1.   13.   32.   40.   16.    0.    0.]
 [  36.   14.    8.    3.    2.    1.    1.   13.   32.   4

In [6]:
def createDataset(df):
    row = int(0.89971*df.shape[0])
    traindf = df[0:row,:]
    testdf = df[row:,:]
    trainY=traindf[:,-1]
    trainX=traindf[:,:-1]
    testY = testdf[:,-1]
    testX = testdf[:,:-1]
    return trainX, trainY, testX, testY

In [7]:
trX, trY, tsX, tsY = createDataset(preppeddata)

In [8]:
print("train X shape", trX.shape)
print("train Y shape", trY.shape)
print("test X shape", tsX.shape)
print("test Y shape", tsY.shape)

train X shape (15636, 24)
train Y shape (15636,)
test X shape (1743, 24)
test Y shape (1743,)


## Training ##

In [9]:
def hyper_parameters(window_sz):
    global units, dropouts, layers, outputunits, input_dim, batch_sz, numepochs
    unitset  = [96, 96, 96, 96]
    layerset = [1, 1, 1, 1, 1]
    dropoutset = [0.4,0.4,0.4]
    
    layers = random.choice(layerset)
    units = np.zeros(layers)
    dropouts = np.zeros(layers)
    batch_sz = 96
    for i in range(0, layers):
        units[i] = random.choice(unitset)
        dropouts[i] = random.choice(dropoutset)
    units[0] = batch_sz
    
    outputunits = 1
    
    numepochs = 10
    
    print(layers)
    print(units)
    print(dropouts)

Try creating network without dropouts. 

In [10]:
def setupNetwork(window_sz):
    global units, dropouts, layers, outputunits
    model = Sequential()

    timestamps = 1
    features = 24

    model.add(keras.layers.LSTM(units=units[0].astype("int64"), return_sequences=False, input_shape=(features, timestamps)))
    #model.add(Dropout(dropouts[0]))
    model.add(Dense(units=batch_sz*2))
    model.add(Activation('relu'))
    model.add(Dense(timestamps))
    model.add(Activation('linear'))

    model.compile(loss='mse', optimizer='rmsprop')
    return model

In [11]:
hyper_parameters(slidingwindow)
model = setupNetwork(slidingwindow)
print(model.summary())

1
[ 96.]
[ 0.4]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 96)                37632     
_________________________________________________________________
dense_1 (Dense)              (None, 192)               18624     
_________________________________________________________________
activation_1 (Activation)    (None, 192)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 193       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 56,449
Trainable params: 56,449
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
def trainModel(m, xtr, ytr, ep, bsz,l=0):
    

    m.fit(xtr, ytr, batch_size=bsz, epochs=ep, validation_split=0.079, verbose=l)
    return m

In [13]:
global batch_sz
print(batch_sz)
modelrest = trainModel(model, trX.reshape(-1,24,1), trY.reshape(-1,1), 1, batch_sz,2)

96
Train on 14400 samples, validate on 1236 samples
Epoch 1/1
14s - loss: 33861.0920 - val_loss: 49550.7258


## Model Evaluation ##

In [14]:
def compute_perf(m, xts, yts):
    ypred = m.predict(xts.reshape(xts.shape[0],24,-1))
    py = np.squeeze(np.reshape(ypred, newshape= [-1,1]), axis=1)
    r2 = mt.r2_score(yts, ypred)
    #print(r2)
    return r2

Run parameter sweep across batch and epoch to find the best parameters that give best R^2 value

In [15]:
def parameter_sweep():
    marr = np.empty((8,5), dtype=Sequential)
    r2 = np.zeros((8,5))
    rowcnt = 0
    colcnt = 0
    print("------------------------------------------------------")
    print("|     R^2    |            epochs                     |")
    print("------------------------------------------------------")
    print("| batch size | ",end="")
    for e in range(2,12,2):
        print(" %3d  | "% e, end="")
    print()
    print("------------------------------------------------------")
    for b in range(24,192,24):
        colcnt = 0
        print("|   %3d      |"% b, end="")
        for e in range(2,12,2):
            mtmp = setupNetwork(slidingwindow)
            marr[rowcnt, colcnt] = trainModel(mtmp, trX.reshape(-1,24,1), trY.reshape(-1,1), e, b, 0)
            r2[rowcnt, colcnt] = compute_perf(marr[rowcnt, colcnt], tsX, tsY)
            
            r = r2[rowcnt, colcnt]
            print(" %0.4f|"% r,end="")
            colcnt += 1
            #print("batch size = ", b, " epochs = ", e, " r^2 = ", r2[rowcnt, colcnt])
        rowcnt +=1
        print()
    print("------------------------------------------------------")

The comparable data using 12 lag features when trained on Azure ML v1 using Boosted decision tree gives R-Square value of 0.82. 

Running parameter sweep across various batch sizes and epochs taks almost 30 mins on 1 GPU based VM. So only run the cell below if you have 30 mins. Else use the data from my run as shown below which shows that batch size with 48 for 10 epochs gives the best result of R^2 being **0.8936** or use 24 batch size with 6 epochs for decent R^2 of **0.8711**. 

![](http://neerajkh.blob.core.windows.net/images/parametersweep_bikerental.PNG)


In [16]:
start = datetime.datetime.now()
print("start of parameter sweep:", start)
#parameter_sweep()
end = datetime.datetime.now()
print("end of parameter sweep:", end)
print("total time = ", end-start)

start of parameter sweep: 2017-09-10 12:45:16.629069
end of parameter sweep: 2017-09-10 12:45:16.630070
total time =  0:00:00.001001


Running parameter sweep also provided hint that we can combine various batch sizes and epochs to get the best results. Here is the one way to do it

In [17]:
def trainWithVaryingBatchAndEpochs(m):
    
    b = 24
    for e in range(2,8,2):      
        m = trainModel(m, trX.reshape(-1,24,1), trY.reshape(-1,1), e, b, 0)
        r2 = compute_perf(m, tsX, tsY)
        print("At end of epoch %d "% e, end="")
        print("using batch size of %d "% b, end="")
        print("the r^2 is %0.4f|"% r2)
    b = 48
    e = 2
    m = trainModel(m, trX.reshape(-1,24,1), trY.reshape(-1,1), e, b, 0)
    print("At end of epoch %d "% e, end="")
    print("using batch size of %d "% b, end="")
    print("the r^2 is %0.4f|"% r2)
    e = 4
    m = trainModel(m, trX.reshape(-1,24,1), trY.reshape(-1,1), e, b, 0)
    print("At end of epoch %d "% e, end="")
    print("using batch size of %d "% b, end="")
    print("the r^2 is %0.4f|"% r2)
    return m
        

In [18]:
mtmp = setupNetwork(slidingwindow)
#mtmp = trainWithVaryingBatchAndEpochs(mtmp)

Now we can try to see if dropouts improve accuracy or not. So we will add dropouts with above configuration and try the new network with dropouts

In [19]:
def setupNetworkWithDropouts(window_sz):
    global units, dropouts, layers, outputunits
    model = Sequential()

    timestamps = 1
    features = 24

    model.add(keras.layers.LSTM(units=units[0].astype("int64"), return_sequences=False, input_shape=(features, timestamps)))
    model.add(Dropout(0.5))
    model.add(Dense(units=batch_sz*2, activation='relu'))
    
    model.add(Dropout(0.5))
    model.add(Dense(timestamps, activation='linear'))
    

    model.compile(loss='mse', optimizer='rmsprop')
    return model

In [20]:
#anothermodel = setupNetworkWithDropouts(slidingwindow)
#anothermodel = trainWithVaryingBatchAndEpochs(anothermodel) 

Now that we have tried all alternatives, progressive batch/epoch, dropout etc., the simplest model that provides good accuracy is with batch size 48 and epoch 10. So for now, we will go with that model. 

For now, let's go with batch size of 48 and 10 epochs with no dropouts.

In [21]:
finalmodel = setupNetwork(slidingwindow)
finalmodel = trainModel(model, trX.reshape(-1,24,1), trY.reshape(-1,1), 10, 48,2)

Train on 14400 samples, validate on 1236 samples
Epoch 1/10
12s - loss: 11884.6534 - val_loss: 20991.9635
Epoch 2/10
12s - loss: 7221.2174 - val_loss: 16770.6625
Epoch 3/10
12s - loss: 5393.9261 - val_loss: 12856.9220
Epoch 4/10
12s - loss: 4482.3544 - val_loss: 11958.8152
Epoch 5/10
13s - loss: 3684.3524 - val_loss: 8890.5063
Epoch 6/10
13s - loss: 3303.2398 - val_loss: 12575.3209
Epoch 7/10
12s - loss: 3030.8193 - val_loss: 6506.9300
Epoch 8/10
12s - loss: 2800.8817 - val_loss: 14724.7015
Epoch 9/10
13s - loss: 2650.7879 - val_loss: 7395.3794
Epoch 10/10
12s - loss: 2568.6371 - val_loss: 6417.3821


In [62]:
print("R^2 = ", compute_perf(finalmodel, tsX, tsY))
finalmodel.save("finalmodel.sav")

R^2 =  0.88287373615


## Operationalization ##

Check for existing **scoring file** and if it exist, remove the scoring file

In [63]:
# remove previous bikescore.py
export_path_base = "bikescore.py"
import os
if os.path.exists(export_path_base):
    print("score file already exist, removing score file")
    os.remove(export_path_base)
else:
    print("no score file found - safe to continue")

score file already exist, removing score file


Write init() and run() functions to score.py file using magic commands

In [72]:
%%writefile bikescore.py

import numpy as np
from numpy import newaxis
import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, TimeDistributed
import pandas as pd
import h5py

def init():
    global trainedmod
    from keras.models import load_model
    
    trainedmod = load_model("finalmodel.sav")

Overwriting bikescore.py


In [73]:
%%writefile -a bikescore.py

def run(npa):
    global trainedmod
    
    if (len(npa.shape) > 1):
        ypred = trainedmod.predict(npa.reshape(npa.shape[0],24,1))
    else:
        ypred = trainedmod.predict(npa.reshape(1,24,1))
    retdf = pd.DataFrame(data={"Scored Values":np.squeeze(np.reshape(ypred, newshape= [-1,1]), axis=1)})
    return str(retdf)

Appending to bikescore.py


In [74]:
%%writefile -a bikescore.py

if __name__ =="__main__":
    init()
    tmpX = np.array([[   4.50000000e+02,   3.53000000e+02,   2.85000000e+02,   3.32000000e+02,
                3.77000000e+02,   2.68000000e+02,   2.18000000e+02,   3.87000000e+02,
                8.34000000e+02,   5.08000000e+02,   1.53000000e+02,   4.20000000e+01,
                4.00000000e+00,   1.00000000e+00,   1.00000000e+01,   1.70000000e+01,
                0.00000000e+00,   4.00000000e+00,   1.00000000e+00,   2.00000000e+00,
                5.80000000e-01,   5.45500000e-01,   6.40000000e-01,   3.28400000e-01],
            [   8.90000000e+02,   4.50000000e+02,   3.53000000e+02,   2.85000000e+02,
                3.32000000e+02,   3.77000000e+02,   2.68000000e+02,   2.18000000e+02,
                3.87000000e+02,   8.34000000e+02,   5.08000000e+02,   1.53000000e+02,
                4.00000000e+00,   1.00000000e+00,   1.00000000e+01,   1.80000000e+01,
                0.00000000e+00,   4.00000000e+00,   1.00000000e+00,   2.00000000e+00,
                5.60000000e-01,   5.30300000e-01,   6.40000000e-01,   3.28400000e-01]])
    
    predY = run(tmpX[0:2,:])
    print(predY)

Appending to bikescore.py


In [75]:
def testsavedmodel():
    
    print("R^2 = ", compute_perf(bikescore.trainedmod, tsX, tsY))

In [76]:
import bikescore
bikescore.init()
testsavedmodel()


R^2 =  0.88287373615


Now we have verified that we have loaded correct model as R^2 value of model saved above is same as the model loaded from the file

In [77]:
print(tsX[0:2,:])

[[  4.50000000e+02   3.53000000e+02   2.85000000e+02   3.32000000e+02
    3.77000000e+02   2.68000000e+02   2.18000000e+02   3.87000000e+02
    8.34000000e+02   5.08000000e+02   1.53000000e+02   4.20000000e+01
    4.00000000e+00   1.00000000e+00   1.00000000e+01   1.70000000e+01
    0.00000000e+00   4.00000000e+00   1.00000000e+00   2.00000000e+00
    5.80000000e-01   5.45500000e-01   6.40000000e-01   3.28400000e-01]
 [  8.90000000e+02   4.50000000e+02   3.53000000e+02   2.85000000e+02
    3.32000000e+02   3.77000000e+02   2.68000000e+02   2.18000000e+02
    3.87000000e+02   8.34000000e+02   5.08000000e+02   1.53000000e+02
    4.00000000e+00   1.00000000e+00   1.00000000e+01   1.80000000e+01
    0.00000000e+00   4.00000000e+00   1.00000000e+00   2.00000000e+00
    5.60000000e-01   5.30300000e-01   6.40000000e-01   3.28400000e-01]]


In [78]:
predY = bikescore.run(tsX[0:5,:])
trueY=str(pd.DataFrame(data={"Actual Values":np.squeeze(np.reshape(tsY[0:5], newshape= [-1,1]), axis=1)}))

In [79]:
for x,y in zip(predY.split('\n'),trueY.split('\n')):
    print(x.strip(), "\t\t\t",y.strip()) 

Scored Values 			 Actual Values
0     742.635132 			 0          890.0
1     780.415833 			 1          788.0
2     570.603760 			 2          513.0
3     389.780273 			 3          387.0
4     276.378601 			 4          283.0


### Generate schema file ###

In [80]:
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
import azureml.api.realtime.services as amlo16n

In [81]:
inputs = {"npa": SampleDefinition(DataTypes.NUMPY, tsX[0:3,:])}
amlo16n.generate_schema(inputs=inputs,
                            filepath="bikeschema.json",
                            run_func=bikescore.run)
amlo16n.generate_main(user_file="bikescore.py", schema_file="bikeschema.json",
                          main_file_name="main.py")

'main.py'

In [33]:
# check score.py and main.py
%pycat bikescore.py


In [34]:
%pycat main.py

## Setup Environment ##
For cluster deployment, you need to setup you environment with ACS, storage, ACR, AppInsights etc. Cluster deployment expects that you have precreated experimentation and model management account through Ibiza as per instructions in the [Instruction Guide](https://github.com/Azure/ViennaDocs/blob/master/Documentation/Installation.md)

You can validate your account using the command shown below

```
C:\Users\neerajkh\Documents\VIenna\BikeForecastNotebook>az ml account modelmanagement list
{
  "created_on": "2017-08-22T02:34:00.837873Z",
  "description": "",
  "id": "/subscriptions/6d976e3d-6278-4645-ab79-cc5f5fe01a49/resourceGroups/amlgrp2/providers/Microsoft.MachineLearningModelManagement/accounts/neerajteam2hosting",
  "location": "eastus2",
  "model_management_swagger_location": "https://eastus2.modelmanagement.azureml.net/api/subscriptions/6d976e3d-6278-4645-ab79-cc5f5fe01a49/resourceGroups/amlgrp2/accounts/neerajteam2hosting/swagger.json?api-version=2017-09-01-preview",
  "modified_on": "2017-08-22T02:34:00.837873Z",
  "name": "neerajteam2hosting",
  "resource_group": "amlgrp2",
  "sku": {
    "capacity": 1,
    "name": "S1"
  },
  "subscription": "6d976e3d-6278-4645-ab79-cc5f5fe01a49",
  "tags": null,
  "type": "Microsoft.MachineLearningModelManagement/accounts"
}
```

Let's set this account to be the default account for registering models and web services.

```
C:\Users\neerajkh\Documents\VIenna\BikeForecastNotebook>az ml account modelmanagement set -n neerajteam2hosting -g amlgrp2
{
  "created_on": "2017-08-22T02:34:00.837873Z",
  "description": "",
  "id": "/subscriptions/6d976e3d-6278-4645-ab79-cc5f5fe01a49/resourceGroups/amlgrp2/providers/Microsoft.MachineLearningModelManagement/accounts/neerajteam2hosting",
  "location": "eastus2",
  "model_management_swagger_location": "https://eastus2.modelmanagement.azureml.net/api/subscriptions/6d976e3d-6278-4645-ab79-cc5f5fe01a49/resourceGroups/amlgrp2/accounts/neerajteam2hosting/swagger.json?api-version=2017-09-01-preview",
  "modified_on": "2017-08-22T02:34:00.837873Z",
  "name": "neerajteam2hosting",
  "resource_group": "amlgrp2",
  "sku": {
    "capacity": 1,
    "name": "S1"
  },
  "subscription": "6d976e3d-6278-4645-ab79-cc5f5fe01a49",
  "tags": null,
  "type": "Microsoft.MachineLearningModelManagement/accounts"
}
```

Once model management account is created, the next step is to create an environment. Users can create K8 based ACS cluster environment using the following commands.

Let's validate if our environment has been created and if so, then let's set it to be the default environment for our implementation.
```
C:\Users\neerajkh\Documents\BikeForecastNotebook>az ml env setup -n amlcluster -c -l eastus2 -g amlgrp2
Subscription set to Channels
Continue with this subscription (Y/n)? Y
Resource group amlgrp2 already exists, skipping creation.
waiting for AAD role to propagate.done
Provisioning compute resources...
Resource creation submitted successfully.
Resources may take 10-20 minutes to be completely provisioned.
To see if your environment is ready to use, run:
  az ml env show -g amlgrp2 -n amlcluster
Once your environment has successfully provisioned, you can set it as your target context using:
  az ml env set -g amlgrp2 -n amlcluster
C:\Users\neerajkh\Documents\BikeForecastNotebook>az ml env list
[
  {
    "Cluster Name": "amlcluster",
    "Cluster Size": 2,
    "Created On": "2017-08-22T04:03:09.147000+00:00",
    "Location": "eastus2",
    "Provisioning State": "Succeeded",
    "Resource Group": "amlgrp2",
    "Subscription": "6d976e3d-6278-4645-ab79-cc5f5fe01a49"
  }
]
C:\Users\neerajkh\Documents\BikeForecastNotebook>az ml env set -n amlcluster -g amlgrp2
Compute set to amlcluster.
```

You are all set with environment


### Create web service ###
Now you can create web service by using command az ml service and passing scoring file, schema file, dependencies file, and model file as shown below


```
C:\Users\neerajkh\Documents\bikeshare1>az ml service create realtime -f bikescore.py -m finalmodel.sav -r python -c .\aml_config\conda_dependencies.yml -s bikeschema.json -l true -n bikews1

Found existing local service with the same name running at http://127.0.0.1:32781/score
Delete existing service and create new service (y/N)? y
Service deleted.
 C:\Users\neerajkh\AppData\Local\Temp\requirementspy1q34aj.txt
 .\aml_config\conda_dependencies.yml
 finalmodel.sav
Successfully registered model
Id: 0acb4b7e57ce40f08bf5057a46ab3897
More information: 'az ml model show -m 0acb4b7e57ce40f08bf5057a46ab3897'
Creating new driver at C:\Users\neerajkh\AppData\Local\Temp\tmpft68nqyb.py
 bikescore.py
 bikeschema.json
Successfully created manifest
Id: 19e49ed5-5952-4338-9334-64000b1ccd27
More information: 'az ml manifest show -i 19e49ed5-5952-4338-9334-64000b1ccd27'
Creating image...................Done.
Image ID: 4497ac1c-3542-4a1f-9dab-3b8e39e77a7f
More details: 'az ml image show -i 4497ac1c-3542-4a1f-9dab-3b8e39e77a7f'
Usage information: 'az ml image usage -i 4497ac1c-3542-4a1f-9dab-3b8e39e77a7f'
[Local mode] Running docker container.
[Local mode] Pulling the image from mlcrpacr10f446252934.azurecr.io/bikews1:7. This may take a few minutes, depending on your connection speed...
[Local mode] Pulling.................................
```

### List web service details ###

You can list web service details along with calling pattern by using the following command

```
C:\Users\neerajkh\Documents\bikeshare1>az ml service list realtime -o table
Name      Id        UpdatedAt            State
--------  --------  -------------------  -------
bikews1   bikews1   2017-09-11T01:52:18  running
mnistws1  mnistws1  2017-09-10T18:49:51  running
```

```
C:\Users\neerajkh\Documents\bikeshare1>az ml service usage realtime -i bikews1
Scoring URL:
    http://127.0.0.1:32783/score

Headers:
    Content-Type: application/json

Swagger URL:
    http://127.0.0.1:32783/swagger.json

Sample CLI command:
    az ml service run realtime -i bikews1 -d "!! YOUR DATA HERE !!"

Sample CURL call:
    curl -X POST -H "Content-Type:application/json" --data "!! YOUR DATA HERE !!" http://127.0.0.1:32783/score

C:\Users\neerajkh\Documents\bikeshare1>curl http://127.0.0.1:32783/swagger.json

{"consumes": ["application/json"], "paths": {"/score": {"post": {"description": "Run web service's model and get the prediction output", "operationId": "RunMLService", "parameters": [{"schema": {"$ref": "#/definitions/ServiceInput"}, "description": "The input payload for executing the real-time machine learning service.", "in": "body", "name": "serviceInputPayload"}], "responses": {"200": {"schema": {"$ref": "#/definitions/ServiceOutput"}, "description": "The service processed the input correctly and provided a result prediction, if applicable."}, "default": {"schema": {"$ref": "#/definitions/ErrorResponse"}, "description": "The service failed to execute due to an error."}}}}, "/": {"get": {"description": "Simple health check endpoint to ensure the service is up at any given point.", "operationId": "ServiceHealthCheck", "responses": {"200": {"schema": {"type": "string"}, "description": "If service is up and running, this response will be returned with the content 'Healthy'", "examples": {"application/json": "Healthy"}}, "default": {"schema": {"$ref": "#/definitions/ErrorResponse"}, "description": "The service failed to execute due to an error."}}}}}, "definitions": {"ServiceInput": {"type": "object", "example": {"npa": [[450.0, 353.0, 285.0, 332.0, 377.0, 268.0, 218.0, 387.0, 834.0, 508.0, 153.0, 42.0, 4.0, 1.0, 10.0, 17.0, 0.0, 4.0, 1.0, 2.0, 0.58, 0.5455, 0.64, 0.3284], [890.0, 450.0, 353.0, 285.0, 332.0, 377.0, 268.0, 218.0, 387.0, 834.0, 508.0, 153.0, 4.0, 1.0, 10.0, 18.0, 0.0, 4.0, 1.0, 2.0, 0.56, 0.5303, 0.64, 0.3284], [788.0, 890.0, 450.0, 353.0, 285.0, 332.0, 377.0, 268.0, 218.0, 387.0, 834.0, 508.0, 4.0, 1.0, 10.0, 19.0, 0.0, 4.0, 1.0, 2.0, 0.56, 0.5303, 0.68, 0.2985]]}, "properties": {"npa": {"type": "array", "items": {"type": "array", "items": {"type": "number", "format": "double"}}}}}, "ErrorResponse": {"type": "object", "properties": {"status_code": {"type": "integer", "format": "int32"}, "message": {"type": "string"}}}, "ServiceOutput": {"type": "object"}}, "schemes": ["https"], "info": {"description": "API specification for the Azure Machine Learning service ML service", "title": "ML service", "version": "1.0"}, "produces": ["application/json"], "swagger": "2.0"}
```

### Calling web service ###


You can call the web service through az ml or curl. Here is the output by calling web service from az ml CLI

```
C:\Users\neerajkh\Documents\bikeshare1>az ml service run realtime -i bikews1 -d "{\"npa\": [[450.0, 353.0, 285.0, 332.0, 377.0, 268.0, 218.0, 387.0, 834.0, 508.0, 153.0, 42.0, 4.0, 1.0, 10.0, 17.0, 0.0, 4.0, 1.0, 2.0, 0.58, 0.5455, 0.64, 0.3284], [890.0, 450.0, 353.0, 285.0, 332.0, 377.0, 268.0, 218.0, 387.0, 834.0, 508.0, 153.0, 4.0, 1.0, 10.0, 18.0, 0.0, 4.0, 1.0, 2.0, 0.56, 0.5303, 0.64, 0.3284], [788.0, 890.0, 450.0, 353.0, 285.0, 332.0, 377.0, 268.0, 218.0, 387.0, 834.0, 508.0, 4.0, 1.0, 10.0, 19.0, 0.0, 4.0, 1.0, 2.0, 0.56, 0.5303, 0.68, 0.2985]]}"
   Scored Values
0     742.635132
1     780.415833
2     570.603760
```